In [ ]:
acu_conn_select()

In [ ]:
xl_env = xl_init_env(nb_name='AptWeeksSuspend', in_sheet_skip_rows=3, in_sheet_use_cols=7, db_name=acu_conn_selector.value)
xl_env

In [ ]:
acu_curs.callproc('P_PROC_SET', ('DBA_SUSP_OW_MOVE_FOR_SWAPS', xl_env['in_file_name'], 'T_SAW_Delete_Insert'))

In [ ]:
# delete records from last run
sql = "DELETE FROM T_SAW WHERE SAW_XL_FNAM = :fnam AND SAW_PROCESSED IS NULL"
acu_curs.execute(sql, fnam=xl_env['in_file_name'])
acu_curs.rowcount

In [ ]:
# insert in_records into T_SAW
current_year = datetime.datetime.now().year
cols = "SAW_YEAR, SAW_CDREF, SAW_OWNER_SNAME, SAW_RSREF, SAW_APREF, SAW_WEEK, SAW_NEXT_OCCY, SAW_WKREF, SAW_XL_FNAM"
coll = cols.split(", ")
rows_inserted = 0
for i, r in xl_env['in_records'].iterrows():
    rsref = r['CLUBID'].replace('BHT', 'BHH').replace('HMT', 'HMC').replace('PBT', 'PBC').replace('PBR', 'PBC')
    apt = str(r['APT'])
    if rsref == 'BHC' and apt[0] == 'J':
        rsref = 'BHH'
    elif rsref == 'BHH' and apt[0] != 'J':
        apt = 'J' + apt
    wk = "{:02}".format(r['WEEK'])  # use {:04} for CPA memberships
    bd = dict(SAW_YEAR=current_year,
              SAW_CDREF=r['MFEEOWNERID'],
              SAW_OWNER_SNAME=r.get('CD_SNAM1') or r['OW_SNAM1'],
              SAW_RSREF=rsref,
              SAW_APREF=apt,
              SAW_WEEK=wk,
              # SAW_NEXT_OCCY=r['YEAR'] if 'YEAR' in r else (r[7] if r.get(7) else current_year + 1),
              SAW_NEXT_OCCY=r['YEAR'] if 'YEAR' in r else current_year + 1,
              SAW_WKREF=apt + '-' + wk,
              SAW_XL_FNAM=xl_env['in_file_name'],
             )
    #print(bd)
    sql = "INSERT INTO T_SAW (" + cols + ") values (:" + ", :".join(coll) + ")"
    acu_curs.execute(sql, **bd)
    rows_inserted += acu_curs.rowcount
acu_conn.commit()
rows_inserted

In [ ]:
sql = """
select SAW_WKREF as Apt_Wk, SAW_RSREF as Resort, SAW_CDREF as Ow_Ref, SAW_OWNER_SNAME as Owner_Name   --T_SAW.*
     , SAW_YEAR as Cxl_Year, SAW_NEXT_OCCY as Next_Occy
     , substr(''
     || case when length(SAW_APREF) not in (4, case SAW_RSREF when 'PBC' then 3 else -1 end)
             then ', SAW_APREF has wrong size' end
     || case when length(SAW_WEEK) not in (2, case SAW_RSREF when 'CPA' then 4 else -1 end)
             then ', SAW_WEEK has wrong size' end
     || case when length(SAW_WKREF) not in (7, case SAW_RSREF when 'PBC' then 6 when 'CPA' then 9 else -1 end)
             then ', SAW_WKREF has wrong size' end
     || case when not exists (select NULL from T_CD where CD_CODE = substr(SAW_CDREF, 1, 7))
             then ', SAW_CDREF is invalid' 
             else case when not exists (select NULL from T_CD
                                         where CD_CODE = substr(SAW_CDREF, 1, 7)
                                           and instr(translate(upper(CD_SNAM1), 'ÁÉÍÓÚÀÈÌÒÙÄËÏÖÜ''', 'AEIOUAEIOUAEIOU`'),
                                                     translate(upper(trim(substr(SAW_OWNER_SNAME, 1, 
                                                                                 case when instr(SAW_OWNER_SNAME, ',') > 0
                                                                                      then instr(SAW_OWNER_SNAME, ',') - 1 
                                                                                      else length(SAW_OWNER_SNAME) end ))),
                                                               'ÁÉÍÓÚÀÈÌÒÙÄËÏÖÜ''', 'AEIOUAEIOUAEIOU`')) > 0) 
                       then ', Owner name discrepancy(CD=' || (select CD_SNAM1 from T_CD
                                                                where CD_CODE = substr(SAW_CDREF, 1, 7)) || ')' end end
     || case when not exists (select NULL from V_OWNED_WEEKS
                               where WK_OWREF = substr(SAW_CDREF, 1, 7) and DW_WKREF = SAW_WKREF)
             then ', Not owned by this client (but by ' || (select WK_OWREF from T_WK where WK_CODE = SAW_WKREF) || ')' end
     || case when not exists (select NULL from T_WK where WK_CODE = SAW_WKREF) then ', SAW_WKREF is invalid'
             else case when not exists (select NULL from T_WK, T_DW where WK_CODE = SAW_WKREF and WK_DWREF = DW_CODE)
                       then ', WK->DW link broken'
                       else case when exists (select NULL from T_WK, T_DW
                                               where WK_CODE = SAW_WKREF and WK_DWREF = DW_CODE
                                                 and (WK_OCCY > SAW_NEXT_OCCY or DW_OCCY > SAW_NEXT_OCCY)) 
                                 then ', Occupancy year greater than ' || SAW_NEXT_OCCY
                                      || ' (WK=' || (select WK_OCCY from T_WK where WK_CODE = SAW_WKREF)
                                      || ' DW=' || (select DW_OCCY from T_WK, T_DW
                                                     where WK_CODE = SAW_WKREF and WK_DWREF = DW_CODE) || ')' 
                            end end end
         , 3) as Discrepancies
  from T_SAW
 where SAW_XL_FNAM = :fnam and SAW_PROCESSED is NULL
   and ( length(SAW_APREF) not in (4, case SAW_RSREF when 'PBC' then 3 else -1 end)
      or length(SAW_WEEK) not in (2, case SAW_RSREF when 'CPA' then 4 else -1 end)
      or length(SAW_WKREF) not in (7, case SAW_RSREF when 'PBC' then 6 when 'CPA' then 9 else -1 end)
      or not exists (select NULL from T_CD where CD_CODE = substr(SAW_CDREF, 1, 7))
      or not exists (select NULL from T_CD where CD_CODE = substr(SAW_CDREF, 1, 7) 
                                            and instr(translate(upper(CD_SNAM1), 'ÁÉÍÓÚÀÈÌÒÙÄËÏÖÜ''', 'AEIOUAEIOUAEIOU`'),
                                                      translate(upper(trim(substr(SAW_OWNER_SNAME, 1, 
                                                                                  case when instr(SAW_OWNER_SNAME, ',') > 0
                                                                                       then instr(SAW_OWNER_SNAME, ',') - 1
                                                                                       else length(SAW_OWNER_SNAME) end ))),
                                                                'ÁÉÍÓÚÀÈÌÒÙÄËÏÖÜ''', 'AEIOUAEIOUAEIOU`')) > 0) 
      or not exists (select NULL from T_WK where WK_CODE = SAW_WKREF)
      or not exists (select NULL from V_OWNED_WEEKS where WK_OWREF = substr(SAW_CDREF, 1, 7) and DW_WKREF = SAW_WKREF)
      or exists (select NULL from T_WK, T_DW where WK_CODE = SAW_WKREF and WK_DWREF = DW_CODE
                                               and (WK_OCCY > SAW_NEXT_OCCY or DW_OCCY > SAW_NEXT_OCCY))
       )
 order by SAW_WKREF
"""
xl_add_sheet_from_sql(xl_env, sql, "Discrep_1", "1. check for correct apt-week and owner data",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select WK_CODE as Apt_Wk, WK_OWREF as Acu_Ow_Ref, substr(SAW_CDREF, 1, 9) as MFee_Ow_Ref
     , WK_ITREF as Inv_Type
     , WK_STATUS
     , LW_STATUS
     , substr(F_STATUS(WK_STATUS), 1, 27) as Short_Wk_Status_Desc
     , substr(F_STATUS(WK_STATUS, WK_CODE, LW_CODE, DW_CODE), 1, 69) as Long_Wk_Status_Desc
     , substr(''
        --|| case when WK_ITREF in ('RE28', 'FI28') then ', INFI Inventory Type Warning' end
        || case when WK_STATUS not in (770, 790) then ', Invalid WK Status' 
                else case when substr(F_STATUS(WK_STATUS, WK_CODE, LW_CODE, DW_CODE), 1, 3) not in ('770', '790')
                          then ', Unexpected WK Status' end
           end
        || case when (nvl(LW_STATUS, 0) in (850, 880, 881)) then ', Invalid LW Status' end
        || case when WK_OWREF <> substr(SAW_CDREF, 1, 7) then ', Different Owner Ref' end
        -- || case when substr(F_STATUS(WK_STATUS, WK_CODE, LW_CODE, DW_CODE), 1, 3) = '770' or WK_STATUS = 770
        --         then ', Financed' end
        , 3) as Discrepancies
  from T_SAW, T_DW, T_WK left outer join T_LW on WK_LWREF = LW_CODE
 where SAW_WKREF = WK_CODE
   and WK_DWREF = DW_CODE
   and SAW_XL_FNAM = :fnam and SAW_PROCESSED is NULL
   and (1=0
     -- FI28 added by Julie - see her email from 01-12-2016 10:09, and on V48/17-04-2018 reclassified error as warning
     --or WK_ITREF in ('RE28', 'FI28') 
     or WK_STATUS not in (770, 790)
     or substr(F_STATUS(WK_STATUS, WK_CODE, LW_CODE, DW_CODE), 1, 3) not in ('770', '790')
     or (nvl(LW_STATUS, 0) in (850, 880, 881))
     or WK_OWREF <> substr(SAW_CDREF, 1, 7)
     )
 order by WK_CODE
 """
xl_add_sheet_from_sql(xl_env, sql, "Discrep_2", 
                      "2. apt-weeks with either invalid week status or inventory types RE28/FI28 (now only as warning) or LW status 850,880,881 or deal status of 770 (completed finance)",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select WK_CODE as Apt_Wk, WK_CEDE_STATUS as Ceding_Status, WK_OWREF as Ow_Ref, WK_POINTS
     , substr((select case when DW_POINTS > 0
                           then 'i' || DW_POINTS end from T_DW where DW_CODE = WK_DWREF), 1, 12) as Ind_Float_Points
     , substr((select f_stragg(DAC_DLREF) from T_DW, T_DAC
                where DW_CODE = WK_DWREF and DW_DLREF = DAC_DLREF and WK_APREF = DAC_APREF and WK_WEEK = DAC_WEEK)
            , 1, 36) as Ceding_Deals
     , substr((select decode(WK_CEDE_STATUS, 2,'Y ', 1,'y', '') || decode(DAC_CODE, NULL,'', 'S') from T_DW, T_DAC
                where DW_CODE = WK_DWREF and DW_DLREF = DAC_DLREF and WK_APREF = DAC_APREF and WK_WEEK = DAC_WEEK)
            , 1, 36) as CEDED_FLAGS
  from T_SAW, T_WK
 where SAW_WKREF = WK_CODE
   and SAW_XL_FNAM = :fnam and SAW_PROCESSED is NULL
   and (WK_CEDE_STATUS <> 0
     -- remove ceded-by-us warning - requested by Julie in her email from 01-10-16 10:47
     --or exists (select NULL from T_DW, T_DAC 
     --            where DW_CODE = WK_DWREF and DW_DLREF = DAC_DLREF and WK_APREF = DAC_APREF and WK_WEEK = DAC_WEEK)
       )
 order by WK_CODE
 """
xl_add_sheet_from_sql(xl_env, sql, "Discrep_3", "3. suspended apt-weeks with RCI Points ceded/in-ceding flag",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select c.CUA_WKREF, c.CUA_CDREF, c.CUA_YEAR, c.CUA_ROREF, c.CUA_STATUS, 'Inv=' || AOWN_ROREF
     , substr(''
           || case when ARO_CODE is not NULL
                   then ', ' || case when ARO_STATUS >= 300 then 'Used' end
                        || 'Apt=' || ARO_APREF || '(' || to_char(ARO_EXP_ARRIVE, 'DD-MON-YY') || ')'
                   when RU_CODE is not NULL
                   then ', Unit=' || RU_ATGENERIC || '@' || RU_RESORT || '(' || to_char(RU_FROM_DATE, 'DD-MON-YY') || ')'
                   else ', Year=' || to_char(c.CUA_YEAR) end
           || case when case when ARO_CODE is not NULL then ARO_EXP_ARRIVE
                             when RU_CODE is not NULL then RU_FROM_DATE
                             else to_date(to_char(sysdate, 'DD-MON') || '-' || to_char(c.CUA_YEAR), 'DD-MON-YYYY') end
                        >= trunc(sysdate)
                   then ', Future ' || case when ARO_CODE is not NULL then 'Apt. Booking' else 'Unit Request' end end
           || case when exists ( select NULL from T_SAW 
                                  where SAW_WKREF = c.CUA_WKREF
                                    and substr(SAW_CDREF, 1, 7) = c.CUA_CDREF
                                    and SAW_NEXT_OCCY <= c.CUA_YEAR
                                    -- ensure processing of current suspension batch records
                                    and SAW_XL_FNAM = :fnam
                                    and SAW_PROCESSED is NULL ) then ', Next Occupancy Year Conflict' end
              , 3) as Notes
  from T_CUA c
       left outer join T_CUA d on d.CUA_TRANS_GROUP = c.CUA_TRANS_GROUP and d.CUA_APT_VALUE_DR > 0 and d.CUA_STATUS <> 120
       left outer join T_RU on RU_UAREF = d.CUA_CODE and RU_STATUS <> 120
       --left outer join T_RH on RH_CODE = RU_RHREF and RH_STATUS <> 120
       left outer join T_ARO on ARO_RHREF = RU_RHREF and ARO_EXP_ARRIVE = RU_FROM_DATE
                            and ARO_EXP_DEPART = RU_FROM_DATE + RU_DAYS and ARO_STATUS <> 120
       left outer join T_AOWN_VIEW on c.CUA_WKREF = AOWN_WKREF and c.CUA_YEAR = AOWN_YEAR 
 where c.CUA_APT_VALUE_CR > 0 
   and c.CUA_STATUS <> 120
   and exists (select * from T_SAW 
                       where SAW_WKREF = c.CUA_WKREF
                         and substr(SAW_CDREF, 1, 7) = c.CUA_CDREF
                         and SAW_NEXT_OCCY <= c.CUA_YEAR
                         -- ensure processing of current suspension batch records
                         and SAW_XL_FNAM = :fnam
                         and SAW_PROCESSED is NULL)
 order by CUA_WKREF, CUA_YEAR
 """
xl_add_sheet_from_sql(xl_env, sql, "Log_4", 
                      "4. list of CUA credits with or without associated debit/booking - which will be cancelled in the suspension code block of this script",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select CUA_WKREF, F_RESORT(CUA_WKREF) as RESORT, CUA_YEAR, CUA_CDREF, CUA_ROREF, CUA_STATUS
     , substr(''
       || case CUA_STATUS when 120 then ', credit cxl' when 123 then ', credit blocked' end
       || case when exists (select NULL from T_SAW
                             where SAW_WKREF = CUA_WKREF and nvl(substr(SAW_LOG_MSG, 1, 3), 'SUS') = 'SUS')
               then (select ', ' || f_stragg(case when SAW_PROCESSED is NULL
                                                  then 'will be suspended (w/ current batch loaded on ' || SAW_LOADED || ')'
                                                  else 'suspended on ' || SAW_PROCESSED || ' with ' || SAW_LOG_MSG end)
                       from T_SAW where SAW_WKREF = CUA_WKREF and nvl(substr(SAW_LOG_MSG, 1, 3), 'SUS') = 'SUS'
                                    and SAW_XL_FNAM = :fnam) end
       || case when F_MFEES_PROTECTED_ACC(CUA_CDREF) = 1 then ', MFee Client Protected' end
       || case when F_MFEES_SUSPENDED_ACC(CUA_CDREF) is not NULL 
               then ', MFee Client Suspended in ' || F_MFEES_SUSPENDED_ACC(CUA_CDREF) end
       , 3) as Notes  
  from T_CUA
 where CUA_APT_VALUE_CR > 0
   and CUA_ROREF in ('rW')       --, 'NN')
   and CUA_STATUS <> 120
   and CUA_YEAR >= to_number((select SAW_NEXT_OCCY from T_SAW
                               where SAW_WKREF = CUA_WKREF 
                                 and substr(SAW_CDREF, 1, 7) = CUA_CDREF
                                 and SAW_XL_FNAM = :fnam and SAW_PROCESSED is NULL))
   -- removed next line and sheet header(down) for Esther (see her email from 9-10-208 14:45)
   --and not exists (select NULL from T_AOWN where WK_CODE = CUA_WKREF and YEAR = CUA_YEAR and AOWN_ROREF = 'RX')
   and exists (select NULL from T_SAW
                where SAW_WKREF = CUA_WKREF and substr(SAW_CDREF, 1, 7) = CUA_CDREF 
                  and SAW_XL_FNAM = :fnam and SAW_PROCESSED is NULL)
   --and (exists (select NULL from T_SAW
   --              where SAW_WKREF = CUA_WKREF and substr(SAW_CDREF, 1, 7) = CUA_CDREF 
   --                and SAW_XL_FNAM = :fnam and SAW_PROCESSED is NULL)
   --     --or F_MFEES_PROTECTED_ACC(CUA_CDREF) = 1
   --     --or instr(F_MFEES_SUSPENDED_ACC(CUA_CDREF), F_RESORT(CUA_WKREF)) > 0
   --    )
   and CUA_EXC_WKREF is NULL and CUA_EXC_YEAR is NULL
   --and CUA_WKREF in ('1114-21', '1203-01')
 order by CUA_WKREF, CUA_YEAR
 """
xl_add_sheet_from_sql(xl_env, sql, "Log_5", 
                      "5. list of rW credits of confiscated apt-weeks, not-internally exchanged",  #, with no RX reservation inventory associated",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select SAW_WKREF, substr(SAW_CDREF, 1, 9) as SAW_CDREF
     , (select f_stragg(AOWN_YEAR || '=' || AOWN_ROREF) from T_AOWN_VIEW
         where AOWN_WKREF = SAW_WKREF and AOWN_YEAR >= SAW_NEXT_OCCY) as InvTypes
  from T_SAW
 where SAW_XL_FNAM = :fnam and SAW_PROCESSED is NULL
 order by SAW_WKREF
 """
xl_add_sheet_from_sql(xl_env, sql, "Log_6", "6. log inventory types of suspended apt-weeks before the suspension",
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
processed = list()
inv_types = dict(BHC='SUBC', BHH='SUBH', HMC='SUHC', PBC='SUPB', CPA='SPHO')
bind_vars = dict(fnam=xl_env['in_file_name'])
sql = """
    select SAW_WKREF, SAW_RSREF, SAW_CDREF, SAW_YEAR, SAW_NEXT_OCCY
      from T_SAW
     where SAW_PROCESSED is NULL        -- hide excluded and already processed aptWks
       and SAW_XL_FNAM = :fnam
     order by SAW_CDREF, SAW_WKREF
"""
to_process = pd.read_sql_query(sql, con=acu_conn, params=bind_vars).to_dict(orient='records')
for idx, row in enumerate(to_process):
    ref_param = prepare_ref_param("")
    params = (ref_param,
              row['SAW_WKREF'],
              inv_types.get(row['SAW_RSREF'], '????'),
              row['SAW_CDREF'][:7],   # current/suspended owner (removing the trailing C from client ref.) 
              row['SAW_YEAR'],
              row['SAW_NEXT_OCCY'],   # occy for resales/nextOwner
              False,                  # NO commit
             )
    # WK_STATUS get set to Completed for CPA memberships and to Available for TS apt-weeks
    named_params = dict(pnWkStatus=790 if row['SAW_RSREF'] == 'CPA' else 890)
    print("{}/{} Params {}+{}".format(idx + 1, len(to_process), params, named_params))
    try:
        acu_curs.callproc('P_SUSPEND_APTWK', params, keywordParameters=named_params)
        row['SAW_LOG_MSG'] = ref_param.getvalue()
    except Exception as ex:
        row['SAW_LOG_MSG'] = "ERE: exception {} processing data {}".format(ex, row)
    
    bind_vars.update(row)
    sql = """
    update T_SAW
       set SAW_LOG_MSG = substr(:SAW_LOG_MSG, 1, 369)
         , SAW_PROCESSED = sysdate
     where SAW_PROCESSED is NULL and SAW_LOG_MSG is NULL
       and SAW_WKREF = :SAW_WKREF
       and SAW_RSREF = :SAW_RSREF
       and SAW_CDREF = :SAW_CDREF 
       and SAW_YEAR = :SAW_YEAR
       and SAW_NEXT_OCCY = :SAW_NEXT_OCCY 
       and SAW_XL_FNAM = :fnam
    """
    #print("Bindvars", bind_vars)
    try:
        acu_curs.execute(sql, **bind_vars)
        acu_conn.commit()
        row['SAW_LOG_MSG'] += "; logged={}".format(acu_curs.rowcount)
    except Exception as ex:
        print("***  EXCEPTION  ***", ex)
        acu_conn.rollback()
        row['SAW_LOG_MSG'] += "; log exception={}".format(ex)
    #print("Row", row)
    processed.append(row)
xl_add_sheet_from_df(xl_env['xl_writer'], pd.DataFrame.from_records(processed), "Log_7", "7. log deal creation")

In [ ]:
sql = """
select nvl(SAW_LOG_MSG, 'CXL: ' || SAW_WKREF || ' of ' || SAW_CDREF || ' in ' || SAW_YEAR
                        || ' for occy ' || SAW_NEXT_OCCY) as LOG_MSG
  from T_SAW
 where SAW_XL_FNAM = :fnam
   and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
   --and trunc(SAW_LOADED) = '28-JUL-2015'
 order by SAW_LOG_MSG, SAW_WKREF
 """
xl_add_sheet_from_sql(xl_env, sql, "Log_8",
                      "8. protocol of PROCESSED apt-weeks - checking suspension deal creation errors (ERR and ERE are ordered onto the front of this result list)",
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select lpad(CUA_CODE, 8) as CUA_CODE, CUA_WKREF, CUA_CDREF, CUA_YEAR, CUA_ROREF, CUA_STATUS
  from T_CUA c
 where CUA_APT_VALUE_CR > 0
   --and CUA_ROREF = 'NN' --not in ('RX', 'rW', 'RW')
   and CUA_STATUS <> 120
   and CUA_YEAR >= to_number((select SAW_NEXT_OCCY from T_SAW
                               where SAW_WKREF = CUA_WKREF and substr(SAW_CDREF, 1, 7) = CUA_CDREF
                                 and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                                 and SAW_XL_FNAM = :fnam))
   and not exists (select NULL from T_CUA d
                    where d.CUA_TRANS_GROUP = c.CUA_TRANS_GROUP and d.CUA_APT_VALUE_DR > 0 and d.CUA_STATUS <> 120)
   and exists (select * from T_SAW, T_WK, T_IT 
                where SAW_WKREF = WK_CODE and WK_ITREF = IT_CODE
                  -- fully confiscated or (if SAW_PROCESSED is NULL) in supension (waiting for owner comitee confirmation)
                  and substr(nvl(SAW_LOG_MSG, 'SUS:'), 1, 4) = 'SUS:'
                  and SAW_WKREF = CUA_WKREF
                  --removed next line to also get old floating bookings (see email from Carol from 08-07-15 9:45):
                  --and SAW_NEXT_OCCY <= CUA_YEAR
                  and substr(SAW_CDREF, 1, 7) = CUA_CDREF
                  -- exclude meanwhile swapped or resold entitlements
                  --and WK_OWREF = case when SAW_PROCESSED is NULL then substr(SAW_CDREF, 1, 7) else IT_OWREF end
                  and (exists (select NULL from V_POWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF)
                     or exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF))
                  -- ensure processing of records that got suspended within the last 3 hours (180 minutes)  
                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                  and SAW_XL_FNAM = :fnam)
 order by CUA_WKREF, CUA_YEAR
"""
xl_add_sheet_from_sql(xl_env, sql, "Log_9",
                      "9. list of cancelled CUA credits for all the suspended apt-weeks/entitlements that NOT got cancelled by the suspension code block",
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
acu_curs.callproc('P_PROC_SET', (None, None, 'CancelResCredits'))
sql = """
update T_CUA c
   set CUA_STATUS = 120
 where CUA_APT_VALUE_CR > 0
   --and CUA_ROREF = 'NN' --not in ('RX', 'rW', 'RW')
   and CUA_STATUS <> 120
   and CUA_YEAR >= to_number((select SAW_NEXT_OCCY from T_SAW
                               where SAW_WKREF = CUA_WKREF and substr(SAW_CDREF, 1, 7) = CUA_CDREF
                                 and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                                 and SAW_XL_FNAM = :fnam))
   and not exists (select NULL from T_CUA d where d.CUA_TRANS_GROUP = c.CUA_TRANS_GROUP and d.CUA_APT_VALUE_DR > 0 
                                              and d.CUA_STATUS <> 120)
   and exists (select * from T_SAW, T_WK, T_IT 
                where SAW_WKREF = WK_CODE and WK_ITREF = IT_CODE
                  -- fully confiscated or (if SAW_PROCESSED is NULL) in supension (waiting for owner comitee confirm)
                  and substr(nvl(SAW_LOG_MSG, 'SUS:'), 1, 4) = 'SUS:'
                  and SAW_WKREF = CUA_WKREF
                  --removed to also get old floating bookings (see email from Carol from 08-07-15 9:45):
                  --and SAW_NEXT_OCCY <= CUA_YEAR
                  and substr(SAW_CDREF, 1, 7) = CUA_CDREF
                  -- exclude meanwhile swapped or resold entitlements
                  --and WK_OWREF = case when SAW_PROCESSED is NULL then substr(SAW_CDREF, 1, 7) else IT_OWREF end
                  and (exists (select NULL from V_POWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF)
                     or exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF))
                  -- ensure processing of records that got suspended within the last 3 hours (180 minutes)  
                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                  and SAW_XL_FNAM = :fnam)
"""
acu_curs.execute(sql, fnam=xl_env['in_file_name'])
acu_conn.commit()
acu_curs.rowcount

In [ ]:
sql = """
select AOWN_CODE, AOWN_WKREF, AOWN_YEAR, AOWN_ROREF
  from T_AOWN_VIEW a
 where 1=1
   and AOWN_ROREF = 'TO' --not in ('RX', 'rW', 'RW', 'SU')
   and AOWN_RSREF <> 'CPA'  -- no need to change for CPA entitlements
   and AOWN_YEAR >= to_number((select SAW_NEXT_OCCY from T_SAW
                                where SAW_WKREF = AOWN_WKREF and substr(SAW_CDREF, 1, 7) = AOWN_CDREF
                                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                                  and SAW_XL_FNAM = :fnam))
   and exists (select * from T_SAW, T_WK w, T_IT 
                where SAW_WKREF = w.WK_CODE and WK_ITREF = IT_CODE
                  -- fully confiscated or (if SAW_PROCESSED is NULL) in supension (waiting for owner comitee confirm)
                  and substr(nvl(SAW_LOG_MSG, 'SUS:'), 1, 4) = 'SUS:'
                  and SAW_WKREF = AOWN_WKREF 
                  and SAW_NEXT_OCCY <= AOWN_YEAR
                  -- exclude meanwhile swapped or resold entitlements
                  --and WK_OWREF = case when SAW_PROCESSED is NULL then substr(SAW_CDREF, 1, 7) else IT_OWREF end
                  and (exists (select NULL from V_POWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF)
                     or exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF))
                  and not exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = substr(SAW_CDREF, 1, 7)
                                                                     and DW_WKREF = SAW_WKREF)
                  -- ensure processing of records that got suspended within the last 3 hours (180 minutes)  
                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                  and SAW_XL_FNAM = :fnam)
 order by AOWN_WKREF, AOWN_YEAR
"""
xl_add_sheet_from_sql(xl_env, sql, "Log_10",
                      "10. list of bulk updates of TO reservation inventory to SU for all the apt-weeks/entitlements",
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
acu_curs.callproc('P_PROC_SET', (None, None, 'ResInvToSU'))
sql = """
update T_AOWN_VIEW a
   set AOWN_ROREF = 'SU'
 where AOWN_ROREF = 'TO' --not in ('RX', 'rW', 'RW', 'SU')
   and AOWN_RSREF <> 'CPA'  -- no need to change for CPA entitlements
   and AOWN_YEAR >= to_number((select SAW_NEXT_OCCY from T_SAW
                                where SAW_WKREF = AOWN_WKREF and substr(SAW_CDREF, 1, 7) = AOWN_CDREF
                                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                                  and SAW_XL_FNAM = :fnam))
   and exists (select * from T_SAW, T_WK w, T_IT 
                where SAW_WKREF = w.WK_CODE and WK_ITREF = IT_CODE
                  -- fully confiscated or (if SAW_PROCESSED is NULL) in supension (waiting for owner comitee confirmation)
                  and substr(nvl(SAW_LOG_MSG, 'SUS:'), 1, 4) = 'SUS:'
                  and SAW_WKREF = AOWN_WKREF 
                  and SAW_NEXT_OCCY <= AOWN_YEAR
                  -- exclude meanwhile swapped or resold entitlements
                  --and WK_OWREF = case when SAW_PROCESSED is NULL then substr(SAW_CDREF, 1, 7) else IT_OWREF end
                  and (exists (select NULL from V_POWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF)
                     or exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF))
                  and not exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = substr(SAW_CDREF, 1, 7)
                                                                     and DW_WKREF = SAW_WKREF)
                  -- ensure processing of records that got suspended within the last 3 hours (180 minutes)  
                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                  and SAW_XL_FNAM = :fnam)
"""
acu_curs.execute(sql, fnam=xl_env['in_file_name'])
acu_conn.commit()
acu_curs.rowcount

In [ ]:
sql = """
select AOWN_CODE, AOWN_WKREF, AOWN_YEAR, AOWN_ROREF
  from T_AOWN_VIEW a
 where 1=1
   and AOWN_ROREF in ('rW', 'RW')
   and AOWN_RSREF <> 'CPA'  -- no need to change for CPA entitlements
   and AOWN_YEAR >= to_number((select SAW_NEXT_OCCY from T_SAW
                                where SAW_WKREF = AOWN_WKREF and substr(SAW_CDREF, 1, 7) = AOWN_CDREF
                                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                                  and SAW_XL_FNAM = :fnam))
   and exists (select * from T_SAW, T_WK w, T_IT 
                where SAW_WKREF = w.WK_CODE and WK_ITREF = IT_CODE
                  -- fully confiscated or (if SAW_PROCESSED is NULL) in supension (waiting for owner comitee confirmation)
                  and substr(nvl(SAW_LOG_MSG, 'SUS:'), 1, 4) = 'SUS:'
                  and SAW_WKREF = AOWN_WKREF 
                  and SAW_NEXT_OCCY <= AOWN_YEAR
                  -- exclude meanwhile swapped or resold entitlements
                  --and WK_OWREF = case when SAW_PROCESSED is NULL then substr(SAW_CDREF, 1, 7) else IT_OWREF end
                  and (exists (select NULL from V_POWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF)
                     or exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF))
                  and not exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = substr(SAW_CDREF, 1, 7)
                                                                     and DW_WKREF = SAW_WKREF)
                  -- ensure processing of records that got suspended within the last 3 hours (180 minutes)  
                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                  and SAW_XL_FNAM = :fnam)
 --)  and AOWN_WKREF in ('6111-21', '6305-14', 'J611-29') --/2015
 order by AOWN_WKREF, AOWN_YEAR
"""
xl_add_sheet_from_sql(xl_env, sql, "Log_11",
                      "11. list of bulk updates of rW/RW reservation inventory to RX for all the apt-weeks/entitlements that are either suspended/fully-confiscated",
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
acu_curs.callproc('P_PROC_SET', (None, None, 'ResInvToRX'))
sql = """
update T_AOWN_VIEW a
   set AOWN_ROREF = 'RX'
 where AOWN_ROREF in ('rW', 'RW')
   and AOWN_RSREF <> 'CPA'  -- no need to change for CPA entitlements
   and AOWN_YEAR >= to_number((select SAW_NEXT_OCCY from T_SAW
                                where SAW_WKREF = AOWN_WKREF and substr(SAW_CDREF, 1, 7) = AOWN_CDREF
                                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                                  and SAW_XL_FNAM = :fnam))
   and exists (select * from T_SAW, T_WK w, T_IT 
                where SAW_WKREF = w.WK_CODE and WK_ITREF = IT_CODE
                  -- fully confiscated or (if SAW_PROCESSED is NULL) in supension (waiting for owner comitee confirmation)
                  and substr(nvl(SAW_LOG_MSG, 'SUS:'), 1, 4) = 'SUS:'
                  and SAW_WKREF = AOWN_WKREF 
                  and SAW_NEXT_OCCY <= AOWN_YEAR
                  -- exclude meanwhile swapped or resold entitlements
                  --and WK_OWREF = case when SAW_PROCESSED is NULL then substr(SAW_CDREF, 1, 7) else IT_OWREF end
                  and (exists (select NULL from V_POWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF)
                     or exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF))
                  and not exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = substr(SAW_CDREF, 1, 7)
                                                                     and DW_WKREF = SAW_WKREF)
                  -- ensure processing of records that got suspended within the last 3 hours (180 minutes)  
                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                  and SAW_XL_FNAM = :fnam
              )
"""
acu_curs.execute(sql, fnam=xl_env['in_file_name'])
acu_conn.commit()
acu_curs.rowcount

In [ ]:
sql = """
select WK_CODE
  from T_WK w
 where WK_CEDE_STATUS <> 0
   and exists (select * from T_SAW, T_IT 
                where SAW_WKREF = w.WK_CODE
                  and IT_CODE = w.WK_ITREF
                  -- fully confiscated or (if SAW_PROCESSED is NULL) in supension (waiting for owner comitee confirmation)
                  and substr(nvl(SAW_LOG_MSG, 'SUS:'), 1, 4) = 'SUS:'
                  -- exclude meanwhile swapped or resold entitlements
                  --and WK_OWREF = case when SAW_PROCESSED is NULL then substr(SAW_CDREF, 1, 7) else IT_OWREF end
                  and (exists (select NULL from V_POWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF)
                     or exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF))
                  -- ensure processing of records that got suspended within the last 3 hours (180 minutes)  
                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                  and SAW_XL_FNAM = :fnam)  
 order by WK_CODE
"""
xl_add_sheet_from_sql(xl_env, sql, "Log_12", "12. list of suspended apt-weeks with RCI Points ceded/in-ceding flag",
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
acu_curs.callproc('P_PROC_SET', (None, None, 'RciPointsUnCeding'))
sql = """
update T_WK w set WK_CEDE_STATUS = 0
 where WK_CEDE_STATUS <> 0
   and exists (select * from T_SAW, T_IT 
                where SAW_WKREF = w.WK_CODE
                  and IT_CODE = w.WK_ITREF
                  -- fully confiscated or (if SAW_PROCESSED is NULL) in supension (waiting for owner comitee confirmation)
                  and substr(nvl(SAW_LOG_MSG, 'SUS:'), 1, 4) = 'SUS:'
                  -- exclude meanwhile swapped or resold entitlements
                  --and WK_OWREF = case when SAW_PROCESSED is NULL then substr(SAW_CDREF, 1, 7) else IT_OWREF end
                  and (exists (select NULL from V_POWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF)
                     or exists (select NULL from V_OWNED_WEEKS x where x.WK_OWREF = IT_OWREF and DW_WKREF = SAW_WKREF))
                  -- ensure processing of records that got suspended within the last 3 hours (180 minutes)  
                  and (sysdate - SAW_PROCESSED) between 0 and 180 / (24*60)
                  and SAW_XL_FNAM = :fnam)
"""
acu_curs.execute(sql, fnam=xl_env['in_file_name'])
acu_conn.commit()
acu_curs.rowcount

In [ ]:
acu_curs.callproc('P_PROC_SET', (None, None, None))
xl_exit_env(xl_env)